In [ ]:


# quick path check
import os
base_path = "/../FIVES A Fundus Image Dataset for AI-based Vessel Segmentation"
print(os.listdir(base_path))   # should show train, test, Quality Assessment.xlsx (or similar)

# load metadata
import pandas as pd
excel_path = os.path.join(base_path, "Quality Assessment.xlsx")
df = pd.read_excel(excel_path)
df.head(), df.shape, df.columns.tolist()


In [ ]:
import pandas as pd
excel_path = "/../FIVES A Fundus Image Dataset for AI-based Vessel Segmentation/Quality Assessment.xlsx"

df_train = pd.read_excel(excel_path, sheet_name="Train")
df_test = pd.read_excel(excel_path, sheet_name="Test")

df_train.head(), df_test.head()


In [ ]:
import os

base_path = "/../FIVES A Fundus Image Dataset for AI-based Vessel Segmentation"
train_img_dir = os.path.join(base_path, "train", "Original")
test_img_dir = os.path.join(base_path, "test", "Original")


In [ ]:
df_train["filename"] = df_train["Number"].astype(str) + "_" + df_train["Disease"].astype(str) + ".png"
df_test["filename"] = df_test["Number"].astype(str) + "_" + df_test["Disease"].astype(str) + ".png"


In [ ]:
df_train["image_path"] = df_train["filename"].apply(lambda x: os.path.join(train_img_dir, x))
df_test["image_path"] = df_test["filename"].apply(lambda x: os.path.join(test_img_dir, x))


In [ ]:
missing_train = [f for f in df_train["image_path"] if not os.path.exists(f)]
missing_test = [f for f in df_test["image_path"] if not os.path.exists(f)]

print("Missing in train:", len(missing_train))
print("Missing in test:", len(missing_test))


In [ ]:
train_mapped = df_train[["image_path", "IC", "Blur", "LC", "Disease"]]
test_mapped = df_test[["image_path", "IC", "Blur", "LC", "Disease"]]


In [ ]:
train_mapped.to_csv("/../train_mapped.csv", index=False)
test_mapped.to_csv("/../test_mapped.csv", index=False)



In [ ]:
print("Train samples:", len(train_mapped))
print("Test samples:", len(test_mapped))

In [ ]:
df_train.head()

In [ ]:
print(train_mapped["Disease"].unique())

In [ ]:
label_map = {"A": 0, "D": 1, "G": 2, "N": 3}
train_mapped["label"] = train_mapped["Disease"].map(label_map)
test_mapped["label"] = test_mapped["Disease"].map(label_map)

In [ ]:
from torchvision import transforms

image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),     # or (256, 256) depending on backbone
    transforms.ToTensor(),
])

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import torch

class RetinaDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Load image
        img = Image.open(row["image_path"]).convert("RGB")
        if self.transform:
            img = self.transform(img)

        # Metadata as tensor
        metadata = torch.tensor(
            [row["IC"], row["Blur"], row["LC"]],
            dtype=torch.float32
        )

        # Label
        label = torch.tensor(row["label"], dtype=torch.long)

        return img, metadata, label

In [ ]:
train_dataset = RetinaDataset(train_mapped, transform=image_transforms)
test_dataset = RetinaDataset(test_mapped, transform=image_transforms)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
images, metadata, labels = next(iter(train_loader))
print("Image batch shape:", images.shape)      # Expect: [batch, 3, H, W]
print("Metadata batch shape:", metadata.shape) # Expect: [batch, 3]
print("Labels shape:", labels.shape)           # Expect: [batch]

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class MultimodalModel(nn.Module):
    def __init__(self, metadata_input_dim, num_classes):
        super(MultimodalModel, self).__init__()

        # Image feature extractor (CNN backbone)
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()  # Remove final layer to get embeddings

        # Metadata MLP
        self.metadata_fc = nn.Sequential(
            nn.Linear(metadata_input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU()
        )

        # Combined classifier
        self.classifier = nn.Sequential(
            nn.Linear(512 + 8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, images, metadata):
        img_features = self.cnn(images)
        meta_features = self.metadata_fc(metadata)
        combined = torch.cat((img_features, meta_features), dim=1)
        out = self.classifier(combined)
        return out

In [ ]:
metadata_input_dim = 3  # since your metadata batch is [16, 3]
num_classes = 4  # change if different

model = MultimodalModel(metadata_input_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
import torch

num_epochs = 5  # you can increase later

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, metadata, labels in train_loader:
        # Add this print statement
        print("Labels before moving to device:", labels)
        # Move to GPU if available
        images = images.to(device)
        metadata = metadata.to(device)
        labels = labels.to(device)


        optimizer.zero_grad()
        outputs = model(images, metadata)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

In [ ]:
# Save
torch.save(model.state_dict(), "/../multimodal_model.pth")

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

In [ ]:
class MultimodalModel(nn.Module):
    def __init__(self, metadata_input_dim, num_classes):
        super(MultimodalModel, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()  # remove final layer

        self.metadata_fc = nn.Sequential(
            nn.Linear(metadata_input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(512 + 8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, images, metadata):
        img_features = self.cnn(images)
        meta_features = self.metadata_fc(metadata)
        combined = torch.cat((img_features, meta_features), dim=1)
        return self.classifier(combined)


In [ ]:
metadata_input_dim = 3
num_classes = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MultimodalModel(metadata_input_dim, num_classes).to(device)

In [ ]:
model.load_state_dict(torch.load("/../multimodal_model.pth", map_location=device))
model.eval()  # important for inference

In [ ]:
import pandas as pd
test_mapped = pd.read_csv("/../test_mapped.csv")

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch

class FundusDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row["image_path"]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        metadata = torch.tensor([row["IC"], row["Blur"], row["LC"]], dtype=torch.float32)
        label = torch.tensor(row["label"], dtype=torch.long)
        return image, metadata, label

In [ ]:
import torchvision.transforms as transforms

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

test_dataset = FundusDataset(test_mapped, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# Add the 'label' column back to test_mapped
label_map = {"A": 0, "D": 1, "G": 2, "N": 3}
test_mapped["label"] = test_mapped["Disease"].map(label_map)

images, metadata, labels = next(iter(test_loader))
images, metadata = images.to(device), metadata.to(device)
outputs = model(images, metadata)
_, predicted = torch.max(outputs, 1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import torch
import numpy as np

model.eval()
all_preds = []
all_labels = []
all_outputs = [] # List to store model outputs (logits)

with torch.no_grad():
    for images, metadata, labels in test_loader:
        images = images.to(device)
        metadata = metadata.to(device)
        labels = labels.to(device)

        outputs = model(images, metadata)

        all_outputs.append(outputs.cpu().numpy()) # Store outputs
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Concatenate all outputs
all_outputs_np = np.concatenate(all_outputs, axis=0)
all_probs = torch.nn.functional.softmax(torch.tensor(all_outputs_np), dim=1).numpy() # Calculate probabilities

# 📊 Classification report
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=['A', 'D', 'G', 'N']))

# 📈 Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

# --- ROC Curves ---
y_true_bin = label_binarize(all_labels, classes=[0, 1, 2, 3])
n_classes = y_true_bin.shape[1]

plt.figure(figsize=(7, 6))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], all_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f"Class {i} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--', lw=1)
plt.title("ROC Curves by Class")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, roc_curve, auc,
    roc_auc_score, precision_recall_curve,
    classification_report
)
from sklearn.preprocessing import label_binarize
import numpy as np
import torch

# Assuming you already have:
# all_labels (actual labels), all_preds (predicted labels) from the previous cell

# Convert logits to probabilities (need to re-run the test set evaluation to get outputs)
# For now, we will use the outputs from the last batch, but for full evaluation,
# you would need to collect all outputs during the test loop.
# Let's assume 'outputs' from the last batch is available for demonstration of plotting.
# If you need plots for the entire test set, you'll need to collect all outputs during evaluation.

# Since we don't have all outputs, we'll skip ROC and Precision-Recall for now and just plot the confusion matrix.
# To plot ROC and PR for the entire test set, you would need to modify the evaluation loop in the previous cell
# to store all outputs (logits) in a list, similar to how all_preds and all_labels are collected.

# --- CONFUSION MATRIX ---
plt.figure(figsize=(6, 5))
cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["A", "D", "G", "N"],
            yticklabels=["A", "D", "G", "N"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# --- ROC CURVES ---
# To plot ROC curves for the entire test set, you would need to collect all model outputs (logits)
# during the evaluation loop in the previous cell and then calculate probabilities from them.
# Example (if you had all_outputs list):
# all_outputs_tensor = torch.cat(all_outputs, dim=0)
# all_probs = torch.nn.functional.softmax(all_outputs_tensor, dim=1).cpu().detach().numpy()
# y_true_bin = label_binarize(all_labels, classes=[0, 1, 2, 3])
# n_classes = y_true_bin.shape[1]

# plt.figure(figsize=(7, 6))
# for i in range(n_classes):
#     fpr, tpr, _ = roc_curve(y_true_bin[:, i], all_probs[:, i])
#     roc_auc = auc(fpr, tpr)
#     plt.plot(fpr, tpr, lw=2, label=f"Class {i} (AUC = {roc_auc:.2f})")

# plt.plot([0, 1], [0, 1], 'k--', lw=1)
# plt.title("ROC Curves by Class")
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.legend()
# plt.show()

# --- PRECISION-RECALL CURVES ---
# Similar to ROC curves, you would need all_probs for this.
# plt.figure(figsize=(7, 6))
# for i in range(n_classes):
#     precision, recall, _ = precision_recall_curve(y_true_bin[:, i], all_probs[:, i])
#     plt.plot(recall, precision, lw=2, label=f"Class {i}")

# plt.title("Precision-Recall Curves")
# plt.xlabel("Recall")
# plt.ylabel("Precision")
# plt.legend()
# plt.show()

# --- AUC Summary ---
# Need all_probs for this.
# auc_macro = roc_auc_score(y_true_bin, all_probs, average='macro')
# auc_weighted = roc_auc_score(y_true_bin, all_probs, average='weighted')
# print(f"Macro AUC: {auc_macro:.3f}, Weighted AUC: {auc_weighted:.3f}")